In [1]:
# things we need for NLP
import nltk
#from nltk.stem.lancaster import LancasterStemmer
from nltk import stem

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
ignore_words = ['?', '', '.', ',', '-', '-ийн', '-ын', '-тай', '-г', '-ийг','-д', '-н', '-ний', '-дээр', 'ч', 'л', 'нь', 'юм', 'бэ', 'вэ', 'юу', 'уу']
my_rule_tuple = ("ы$|ий$|ыг$|ийг$|ны$|ний$|лаа$|лээ$|лоо$|лөө$|даа$|дээ$|доо$|дөө$|уулж$|үүлж$|гдах$|гдэх$|гдох$|гдөх$|лалт$|лэлт$|лолт$|лөлт$|дахад$|дэхэд$|доход$|дөхөд$|уудыг$|үүдийг$|нүүдийг$|нуудыг$|чихлаа$|чихлээ$|чихлоо$|чихлөө$|ид$|ын$|ийн$|уудын$|үүдийн$|тай$|той$|төй$|тэй$|лал$|лэл$|лол$|лөл$|аар$|ээр$|оор$|өөр$|лах$|лэх$|лох$|лөх$|ахад$|эхэд$|оход$|өхөд$|лалд$|лэлд$|лөлд$|лолд$|ууд$|үүд$|даг$|дэг$|дог$|дөг$|ддаг$|ддэг$|ддог$|ддөг$|даж$|дэж$|дож$|дөж$|сэн$|сан$|сөн$|сон$|гах$|гэх$|гох$|гөх$|рах$|рэх$|рох$|рөх$|иас$|иэс$|иос$|иөс$|нд$|нт$|ад$|эд$|од$|өд$|ааж$|ээж$|оож$|өөж$|лаар$|лээр$|лоор$|лөөр$|аг$|эг$|ог$|өг$|уг$|өөг$|ээг$|оог$|үүг$|ууг$|бар$|бэр$|бор$|бөр$|бур$|бүр$|раа$|рээ$|рөө$|рүү$|роо$|руу$|аас$|ээс$|оос$|өөс$|аарх$|ээрх$|оорх$|уурх$|үүрх$|өөрх$|аад$|ээд$|ууд$|үүд$|оод$|өөд$|нууд$|нүүд$|дсан$|дсэн$|дсөн$|дсон$|дах$|дэх$|дох$|дөх$|хлаа$|хлээ$|хлоо$|хлөө$|лахдаа$|лэхдээ$|лохдоо$|лөхдөө$|лдаг$|лдэг$|лж$|аагүй$|ээгүй$|оогүй$|өөгүй$|даггүй$|дэггүй$|доггүй$|дөггүй$|нуудийнхаа$|нүүдийнхээ$|гүй$")
#stemmer = LancasterStemmer()
stemmer = stem.RegexpStemmer(my_rule_tuple, min=6)

# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.4.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("Барааны төлөв гэж юу вэ?", words)
#print (p)
#print (classes)

In [6]:
# load our saved model
model.load('./model.tflearn')

In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.45

def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [8]:
classify('Барааны төлөв гэж юу вэ?')

[('Барааны төлөв гэж юу вэ', 0.9999997615814209)]

In [9]:
response('Барааны төлөв гэж юу вэ?')

Бараа бүртгэл дэлгэцийн Төлөв талбар нь:


In [10]:
classify('Бэлнээр төлбөр авдаг уу?')

[('УЧҮ  -г олон хүмүүс үнэлж болох уу?', 0.5863404273986816)]

In [11]:
response('Барааны ангилал гэж юу вэ?')

Идэвхтэй - Барааг идэвхтэй болгосноор худалдан авах, орлого, зарлагын гүйлгээнд ашиглагдах боломжтой болно.


In [12]:
classify('Барааны ангилал гэж яг юу вэ?')

[('Барааны ангилал гэж юу вэ', 0.9999946355819702)]

In [17]:
response('Барааны ангилал ')

Идэвхтэй - Барааг идэвхтэй болгосноор худалдан авах, орлого, зарлагын гүйлгээнд ашиглагдах боломжтой болно.


In [18]:
classify('Барааны ангилал ')

[('Барааны ангилал гэж юу вэ', 0.9999978542327881)]

In [22]:
classify('Барааны ангилал гэж яг юуг хэлэх вэ?')

[('Барааны ангилал гэж юу вэ', 0.999996542930603)]

In [23]:
classify('Бараа материалын ангилал гэж яг юуг хэлэх вэ')

[('Барааны ангилал гэж юу вэ', 0.9788206815719604)]

In [24]:
classify('Бараа материалын ангилал гэж яг юуг хэлэх вэ')

[('Барааны ангилал гэж юу вэ', 0.9788206815719604)]

In [25]:
# clear context
response("Hi there!", show_details=True)

context: 
tag: greeting
Тавтай морилно уу! Хэрэглэгчтэй харилцах автомат ЧатБот байна, та асуултаа асууна уу.


In [26]:
message = "Бараа материалын ангилал гэж яг юуг хэлэх вэ"
result = classify(message)
print (result)

[('Барааны ангилал гэж юу вэ', 0.9788206815719604)]


In [27]:
message = "RMA гэж юу"
result = classify(message)
response(message, show_details=True)

print (len(result))

if len(result) > 0 :
    question = result[0][0]
    confidence = result[0][1]
else :
    confidence = 0

#message = message, " conf: ", confidence

if confidence > 0.998 :
    print(response(message), ' ', confidence)
else :
    print('Уучлаарай, таны асуултанд сайн тохирох хариултыг олсонгүй. Та дахин асууна уу.', ' ', confidence)

tag: Олон валютын модуль юу хийдэг вэ?
Олон төрлийн валют ашигладаг байгууллагуудын хувьд энэхүү дэлгэцийг ашиглана. Олон валют хэсэгт валюттай холбоотой бүх төрлийн мэдээлэл бүртгэгдэнэ
1
Олон төрлийн валют ашигладаг байгууллагуудын хувьд энэхүү дэлгэцийг ашиглана. Олон валют хэсэгт валюттай холбоотой бүх төрлийн мэдээлэл бүртгэгдэнэ
None   0.9998395442962646


In [28]:
#test_questions = open("test_questions.txt", "r")
min_confidence = 1.1
min_confidence_question = ''
total_question = 0
total_confidence = 0

for i in intents['intents']:
    # find a tag matching the first result
    patterns = i['patterns'];
    for pattern in patterns:
        result = classify(pattern)
        confidence = result[0][1]
        question = result[0][0]
        total_question = total_question + 1
        total_confidence = total_confidence + confidence
        
        print (total_question, ". ", pattern, " ", confidence, " ", question)
        if min_confidence > confidence:
            min_confidence = confidence
            min_confidence_question = pattern

print ("min confidence: ", min_confidence, " ", min_confidence_question)
print ("average confidence: ", total_confidence / total_question)
#for line in test_questions:
    #line = line.replace(",", " ")
    #line = line.replace("\"", "")
    #line = line.lstrip()
    ##print('*', line)
    ##response(line, show_details=True)
    #print (classify(line), " ", line)

#test_questions.close()

1 .  Hi   1.0   greeting
2 .  hello   1.0   greeting
3 .  bn uu   1.0   greeting
4 .  Сайн байна уу   1.0   greeting
5 .  байна уу   0.9998431205749512   greeting
6 .  Өдрийн мэнд   1.0   greeting
7 .  Өглөөний мэнд   1.0   greeting
8 .  оройн мэнд   1.0   greeting
9 .  хүн байна уу   0.9999855756759644   greeting
10 .  Bye   0.9999995231628418   goodbye
11 .  cy   0.9999995231628418   goodbye
12 .  Баяртай   0.9999996423721313   goodbye
13 .  see you   0.9999998807907104   goodbye
14 .  Goodbye   0.9999996423721313   goodbye
15 .  good bye   1.0   goodbye
16 .  Баярлалаа   0.9999997615814209   thanks
17 .  Thanks   0.9999996423721313   thanks
18 .  За баярлалаа   1.0   thanks
19 .  гялайлаа   0.9999995231628418   thanks
20 .  Thank you   0.9999997615814209   thanks
21 .  Танайх хэдээс хэдэн цаг хүртэл ажилладаг вэ?   0.9999998807907104   hours
22 .  Хэдэн цаг хүртэл ажилладаг вэ?   0.9999649524688721   hours
23 .  Хэд хүртэл ажиллах вэ?   0.9999990463256836   hours
24 .  Supply Chain 